## **Sequence-to-sequence learning : EN to FR translation**

### **1. Small_vocab translation**

In [1]:
import os

def load_data(path):

    # Nombre maximum de lignes à renvoyer
    max_lines = 140000
    
    input_file = os.path.join(path)
    with open(input_file, "r",  encoding="utf-8") as f:
        data = f.read()
        
    # On convertit les majuscules en minulcule
    data = data.lower()
    
    data = data.split('\n')
    return data[:min(len(data),max_lines)]

#Chargement des textes dans les 2 langues (max lignes = max_lines)
txt_en = load_data('data/small_vocab_en')
txt_fr = load_data('data/small_vocab_fr')

text_pairs = []
for line in range(len(txt_en)):
    txt_fr[line]=txt_fr[line].replace('à', 'a')
    text_pairs.append((txt_en[line], "[start] " + txt_fr[line] + " [end]" ))
    

In [2]:
import random
print(random.choice(text_pairs))

('india is dry during november , and it is usually quiet in february .', "[start] l' inde est sec au mois de novembre , et il est généralement calme en février . [end]")


In [3]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

**Vectorizing the English and French text pairs**

In [4]:
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 30

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_french_texts = [pair[1] for pair in train_pairs]
# source_vectorization.adapt(train_english_texts)
# target_vectorization.adapt(train_french_texts)
english_texts = [pair[0] for pair in text_pairs]
french_texts = [pair[1] for pair in text_pairs]
source_vectorization.adapt(english_texts)
target_vectorization.adapt(french_texts)

**Preparing datasets for the translation task**

In [5]:
batch_size = 64

def format_dataset(eng, fra):
    eng = source_vectorization(eng)
    fra = target_vectorization(fra)
    return ({
        "english": eng,
        "french": fra[:, :-1],
    }, fra[:, 1:])

def make_dataset(pairs):
    eng_texts, fra_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    fra_texts = list(fra_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, fra_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [6]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['french'].shape: {inputs['french'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 30)
inputs['french'].shape: (64, 30)
targets.shape: (64, 30)


### **2. Sequence-to-sequence learning with RNNs**

**GRU-based encoder**

In [7]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

**GRU-based decoder and the end-to-end model**

In [8]:
past_target = keras.Input(shape=(None,), dtype="int64", name="french")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

**Training our recurrent sequence-to-sequence model**

In [9]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15
1508/1508 [==============================] - 124s 75ms/step - loss: 0.3095 - accuracy: 0.8094 - val_loss: 0.0375 - val_accuracy: 0.9768
Epoch 2/15
1508/1508 [==============================] - 112s 74ms/step - loss: 0.0317 - accuracy: 0.9811 - val_loss: 0.0201 - val_accuracy: 0.9878
Epoch 3/15
1508/1508 [==============================] - 112s 74ms/step - loss: 0.0203 - accuracy: 0.9880 - val_loss: 0.0149 - val_accuracy: 0.9911
Epoch 4/15
1508/1508 [==============================] - 112s 74ms/step - loss: 0.0157 - accuracy: 0.9908 - val_loss: 0.0133 - val_accuracy: 0.9923
Epoch 5/15
1508/1508 [==============================] - 112s 74ms/step - loss: 0.0135 - accuracy: 0.9921 - val_loss: 0.0115 - val_accuracy: 0.9932
Epoch 6/15
1508/1508 [==============================] - 112s 74ms/step - loss: 0.0118 - accuracy: 0.9930 - val_loss: 0.0108 - val_accuracy: 0.9938
Epoch 7/15
1508/1508 [==============================] - 112s 74ms/step - loss: 0.0108 - accuracy: 0.9936 - val_loss: 0

**Saving of the trained RNN model**

In [10]:
seq2seq_rnn.save("data/saved-fra-eng/seq2seq_rnn-fra-en-model.h5")

**Loading of the trained RNN model**

In [11]:
seq2seq_rnn = keras.models.load_model("data/saved-fra-eng/seq2seq_rnn-fra-en-model.h5")

**Translating new sentences with our RNN encoder and decoder**

In [18]:
import numpy as np
fra_vocab = target_vectorization.get_vocabulary()
fra_index_lookup = dict(zip(range(len(fra_vocab)), fra_vocab))
max_decoded_sentence_length = 30

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence], verbose=0)
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = fra_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence[8:-6]

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print("EN  ",input_sentence)
    print("FR->",decode_sequence(input_sentence))

-
EN   he is driving a big blue truck .
FR-> il est au volant d un gros camion bleu
-
EN   india is never quiet during april , but it is never beautiful in winter .
FR-> l inde est jamais tranquille en avril mais il est beau jamais en hiver
-
EN   his favorite fruit is the peach , but our favorite is the grapefruit .
FR-> son fruit préféré est la pêche mais notre préféré est le pamplemousse
-
EN   she likes that rusty green car .
FR-> elle aime cette voiture verte rouillée
-
EN   he is driving that big yellow truck .
FR-> il conduit ce grand camion jaune
-
EN   india is freezing during may , but it is never snowy in august .
FR-> l inde est le gel en mai mais jamais de neige en août
-
EN   china is beautiful during march , and it is wet in december .
FR-> chine est belle en mars et il est humide en décembre
-
EN   paris is freezing during autumn , and it is usually rainy in february .
FR-> paris est le gel a lautomne et il est généralement pluvieux en février
-
EN   she likes peaches ,

### **3. Sequence-to-sequence learning with Transformer**

#### The Transformer decoder

**The `TransformerDecoder`**

In [7]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

#### Putting it all together: A Transformer for machine translation

**PositionalEmbedding layer**

In [8]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**End-to-end Transformer**

In [9]:
from keras_nlp.layers import TransformerEncoder
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim, name='positionalEmbeddingEnglish')(encoder_inputs)
encoder_outputs = TransformerEncoder(dense_dim, num_heads, name='encoder_outputs')(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="french")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim, name='positionalEmbeddingFrench')(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads,  name='transformerDecoder')(x, encoder_outputs)
x = layers.Dropout(0.5, name='dropout')(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax", name='decoder_outputs')(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

Using TensorFlow backend


**Training the sequence-to-sequence Transformer**

In [10]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
transformer.fit(train_ds, epochs=15, validation_data=val_ds, callbacks=[stop_early])

Epoch 1/15
1508/1508 [==============================] - 94s 59ms/step - loss: 0.1608 - accuracy: 0.9202 - val_loss: 0.0363 - val_accuracy: 0.9763
Epoch 2/15
1508/1508 [==============================] - 89s 59ms/step - loss: 0.0382 - accuracy: 0.9770 - val_loss: 0.0265 - val_accuracy: 0.9833
Epoch 3/15
1508/1508 [==============================] - 90s 60ms/step - loss: 0.0300 - accuracy: 0.9823 - val_loss: 0.0223 - val_accuracy: 0.9864
Epoch 4/15
1508/1508 [==============================] - 90s 60ms/step - loss: 0.0253 - accuracy: 0.9851 - val_loss: 0.0204 - val_accuracy: 0.9874
Epoch 5/15
1508/1508 [==============================] - 90s 60ms/step - loss: 0.0220 - accuracy: 0.9871 - val_loss: 0.0178 - val_accuracy: 0.9891
Epoch 6/15
1508/1508 [==============================] - 91s 60ms/step - loss: 0.0192 - accuracy: 0.9888 - val_loss: 0.0159 - val_accuracy: 0.9906
Epoch 7/15
1508/1508 [==============================] - 91s 60ms/step - loss: 0.0170 - accuracy: 0.9901 - val_loss: 0.0145 -

**Saving of the trained Transformer model**

In [12]:
for i in range(len(transformer.weights)):
    transformer.weights[i]._handle_name = transformer.weights[i].name + "_" + str(i)
    
transformer.save("data/saved-fra-eng/transformer-fra-en-model.h5", include_optimizer=False)
transformer.save_weights("data/saved-fra-eng/transformer-fra-en-model.weights.h5")


**Loading of the trained Transformer model**

In [13]:
from keras_nlp.layers import TransformerEncoder
transformer = keras.models.load_model(
    "data/saved-fra-eng/transformer-fra-en-model.h5",
    custom_objects={"PositionalEmbedding": PositionalEmbedding, "TransformerDecoder": TransformerDecoder},
)
transformer.load_weights("data/saved-fra-eng/transformer-fra-en-model.weights.h5")

**Translating new sentences with our Transformer model**

In [19]:
import numpy as np
fra_vocab = target_vectorization.get_vocabulary()
fra_index_lookup = dict(zip(range(len(fra_vocab)), fra_vocab))
max_decoded_sentence_length = 30

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = fra_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence[8:-6]

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print("EN  ",input_sentence)
    print("FR->",decode_sequence(input_sentence))

-
EN   she likes mangoes , apples , and bananas .
FR-> elle aime les mangues les pommes et les bananes
-
EN   she was driving that big red automobile .
FR-> elle conduisait cette grande voiture rouge
-
EN   new jersey is usually rainy during february , but it is sometimes quiet in april .
FR-> new jersey est généralement pluvieux en février mais il est parfois calme en avril
-
EN   california is usually rainy during may , and it is sometimes chilly in april .
FR-> californie est généralement pluvieux au mois de mai et il est parfois frisquet en avril
-
EN   france is never pleasant during june , and it is usually freezing in november .
FR-> la france est jamais agréable en juin et il gèle habituellement en novembre
-
EN   paris is snowy during summer , and it is sometimes beautiful in spring .
FR-> paris est neigeux pendant l été et il est parfois beau au printemps
-
EN   she disliked that big white automobile .
FR-> elle naimait pas cette grande voiture blanche
-
EN   the orange is ou

In [15]:
input_sentence = "india is never cold during winter , but it is beautiful in autumn"
print("FR->",decode_sequence(input_sentence))

FR-> inde ne fait jamais froid pendant l hiver mais il est beau a l automne
